# Calculate SPI and SPEI indicators from various input datasets
# TODO choose base period! Probably don't actually want to fit on everything. On the other hand, choosing post 1980 isn't very good either because change has already happened
## Datasets

CRU monthly rainfall and mean temperatures

ECMWF ERA Interim

- The base period from ERA Interim is rather short, so prefer CRU for now


In [41]:
%matplotlib inline

from pathlib import Path
from datetime import date

import numba
import rasterio
import numpy as np
import scipy as sp
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

from scipy import stats
from tqdm import tnrange, tqdm_notebook
from numba import float64, int64, jit, prange


from precipitation_indices import indices, compute
import weather_ecmwf
import population_tools

from config import DATA_SRC, WEATHER_SRC


def spi3_ufunc(data, calibration_year_initial, calibration_year_final):
    data_start_year = np.asscalar(data['time.year'][0])
    return indices.spi_pearson(data, 3, data_start_year, calibration_year_initial, calibration_year_final)
    
#     return indices.spi_gamma(data, 3)


def spi6_ufunc(data):
    data_start_year = np.asscalar(data['time.year'][0])
    return indices.spi_pearson(data, 6, data_start_year, calibration_year_initial, calibration_year_final)


def spi12_ufunc(data):
    data_start_year = np.asscalar(data['time.year'][0])
    return indices.spi_pearson(data, 12, data_start_year, calibration_year_initial, calibration_year_final)

## Apply SPI calculation to CRU monthly precipitation data

In [42]:
cru_ppt = xr.open_mfdataset(str(WEATHER_SRC / 'cru/cru_ts4.01.1901.2016.*.dat.nc'), 
                        chunks={'lat':30, 'lon':30},
                        drop_variables=['stn', 'tmn'], engine='scipy', lock=False)

In [ ]:
spi3 = xr.apply_ufunc(spi3_ufunc, cru_ppt.pre, 
                      input_core_dims=[['time']], 
                      output_core_dims=[['time']], 
                     vectorize=True, dask='parallelized', output_dtypes=[np.float])

In [ ]:
spi3.to_netcdf(DATA_SRC / 'lancet' / 'spi3_1901_2016_cru.nc')

In [44]:
spi6 = xr.apply_ufunc(spi6_ufunc, cru_ppt.pre, 1901, 2000,
                      input_core_dims=[['time']], 
                      output_core_dims=[['time']], 
                     vectorize=True, dask='parallelized', 
                      output_dtypes=[np.float])
spi6.name = 'spi6'

In [45]:
spi6.to_netcdf(DATA_SRC / 'lancet' / 'spi6_1901_2016_cru.nc')

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
spi12 = xr.apply_ufunc(spi6_ufunc, cru_ppt.pre, 
                       input_core_dims=[['time']], 
                       output_core_dims=[['time']], 
                     vectorize=True, dask='parallelized', output_dtypes=[np.float])
spi12.name = 'spi12'

In [ ]:
spi12.to_netcdf(DATA_SRC / 'lancet' / 'spi6_1901_2016_cru.nc')

## Apply SPI calculation to ECMWF monthly precipitation data

In [26]:
PPT_FOLDER = WEATHER_SRC / 'ecmwf'/ 'monthly_precipitation' / '*_precipitation.nc'
era_ppt = xr.open_mfdataset(str(PPT_FOLDER), 
                            engine='scipy', lock=False)

In [32]:
era_ppt = era_ppt.load()
era_ppt = era_ppt.chunk({'latitude':60, 'longitude':60, 'time': -1})

In [28]:
spi6 = xr.apply_ufunc(spi6_ufunc, era_ppt.tp, 
                      input_core_dims=[['time']], 
                      output_core_dims=[['time']], 
                      vectorize=True, dask='parallelized', output_dtypes=[np.float])
spi6.name = 'spi6'

/Users/jonathanchambers/miniconda3/envs/lancet/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/jonathanchambers/miniconda3/envs/lancet/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/jonathanchambers/miniconda3/envs/lancet/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1738: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0
/Users/jonathanchambers/miniconda3/envs/lancet/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1904: RuntimeWarning: invalid value encountered in greater
  cond1 = (0 < q) & (q < 1)
/Users/jonathanchambers/miniconda3/envs/lancet/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1904: RuntimeWarning: invalid value encountered in less
  cond1 = (0 

In [30]:
spi6.to_netcdf(DATA_SRC / 'lancet' / 'spi6_1980_2017_era.nc')

In [33]:
spi12 = xr.apply_ufunc(spi12_ufunc, era_ppt.tp, 
                      input_core_dims=[['time']],
                      output_core_dims=[['time']], 
                      vectorize=True, dask='parallelized', output_dtypes=[np.float])
spi12.name = 'spi12'

In [34]:
spi12.to_netcdf(DATA_SRC / 'lancet' / 'spi12_1980_2017_era.nc')

/Users/jonathanchambers/miniconda3/envs/lancet/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/jonathanchambers/miniconda3/envs/lancet/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/jonathanchambers/miniconda3/envs/lancet/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1738: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0
/Users/jonathanchambers/miniconda3/envs/lancet/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1904: RuntimeWarning: invalid value encountered in greater
  cond1 = (0 < q) & (q < 1)
/Users/jonathanchambers/miniconda3/envs/lancet/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1904: RuntimeWarning: invalid value encountered in less
  cond1 = (0 

# Calculate SPEI

## Apply SPEI calculation to CRU monthly precipitation data

In [ ]:
# def spei6_ufunc(precips_mm, temperatures, lat):
#     return indices.spei_gamma(precips_mm, 6, temps_celsius=temperatures, data_start_year=1901, latitude_degrees=lat)

# -------------------------------------------------------
# Manually ufunc'ed versions of SPEI index calculations
# -------------------------------------------------------

@jit(nogil=True, parallel=True)
def spei6_ufunc(precips_mm, temperatures, lat):
    months_scale=6
    spei_out = np.empty_like(precips_mm)
    for i in prange(precips_mm.shape[0]):
        latitude_degrees = lat[i]
        for j in prange(precips_mm.shape[1]):
            p_mm = precips_mm[i,j,:]
            t_celsius = temperatures[i,j,:]
            spei_out[i, j,:] = indices.spei_gamma(p_mm, months_scale, temps_celsius=t_celsius, 
                                                  data_start_year=1901, latitude_degrees=latitude_degrees)
    return spei_out


@jit(nogil=True, parallel=True)
def spei3_ufunc(precips_mm, temperatures, lat):
    months_scale=3
    spei_out = np.empty_like(precips_mm)
    for i in prange(precips_mm.shape[0]):
        latitude_degrees = lat[i]
        for j in prange(precips_mm.shape[1]):
            p_mm = precips_mm[i,j,:]
            t_celsius = temperatures[i,j,:]
            spei_out[i, j,:] = indices.spei_gamma(p_mm, months_scale, temps_celsius=t_celsius, 
                                                  data_start_year=1901, latitude_degrees=latitude_degrees)
    return spei_out



In [ ]:
spei6 = xr.apply_ufunc(spei6_ufunc, cru_ppt.pre, cru_ppt.tmp, cru_ppt.lat, 
                       input_core_dims=[['time'],['time'],[]],
                       output_core_dims=[['time']], 
                       dask='parallelized', 
                       output_dtypes=[np.float])

In [ ]:
spei6.to_netcdf(DATA_SRC / 'lancet' / 'spei6_1901_2016_cru.nc')

In [ ]:
spei3 = xr.apply_ufunc(spei3_ufunc, cru_ppt.pre, cru_ppt.tmp, cru_ppt.lat, 
                       input_core_dims=[['time'],['time'],[]],
                       output_core_dims=[['time']], 
                       dask='parallelized', 
                       output_dtypes=[np.float])

In [ ]:
spei3.to_netcdf(DATA_SRC / 'lancet' / 'spei3_1901_2016_cru.nc')